In [1]:
import sys
from common import commons
home = commons.home
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit,cross_validate
from common import Convert_To_Normal_Dist as ctnd
from features_selection import TTest as tt
from matplotlib import pyplot as plt
from features_selection import CorrFeatureSelector as cfs
from simulation import AutoencoderSimulator as AS
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from features_selection import Feature_Selection as FS
from common import DataScaler as ds
from log import Logger
from feature_engineering import Sparse_Autoencoder_class as sac
from hyperopt import fmin,tpe,hp, STATUS_OK,Trials
from sklearn.preprocessing import StandardScaler
import math
from features_selection import TTest

/home/ec2-user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#--------------------------------------------------------------------- 
def normalize_feature(all_features,dataset='AD_CpG'):
    count_thresh = 10
    unique_value_counts = [len(all_features[col].unique()) for col in all_features.columns[2:]] #ignore chr and coordinate
    normalize_all_features = all_features.copy()
    mappings = pd.DataFrame()
    for col,num in zip(all_features.columns[2:],unique_value_counts):
        if num >= count_thresh:
            print(col)
            atn = ctnd.AnyToNormal(col)
            mapping = atn.transform(normalize_all_features)
            mappings[mapping.columns[0]] = mapping.ix[:,0]
            mappings[mapping.columns[1]] = mapping.ix[:,1]
        else: 
            continue
    h5s = pd.HDFStore(home+'data/'+dataset+'/normalize_mapping','w')
    h5s['mappings'] = mappings
    h5s['normal_features'] = normalize_all_features
    h5s.close()
    return home+'data/'+dataset+'/normalize_mapping',normalize_all_features,mappings
   
#-------------------------------------------------------------------------------
def data_selection(data,classes=[0,1,-1],combine=False,*args):              #which classes of data to keep
    if not isinstance(data,pd.DataFrame):
        cols = args
        data = pd.DataFrame(data,columns=cols)    
    ret_data = data.query('label in @classes')
    for i,label in enumerate(classes):
        ret_data['label'][ret_data['label']==label] = i
    
    if len(classes)>2 and combine==True:
        ret_data['label'] = ret_data['label'].where(ret_data['label']==0,1)
    return ret_data
        
#------------------------------------------------------------------------------                  
 
def TSNEPlot(data,class_labels,param_map):##param_map like 1:('r','o','80')
    data1 = data.reset_index()
    tsne = TSNE(n_components=3,random_state=91)
    feature_reduced = tsne.fit_transform(data1.drop(['label','index'],axis=1))
    fig = plt.figure(figsize=(18,15))
    ax = fig.gca(projection='3d')
    for i in class_labels:
        c,marker,size = param_map[i]
        print(c,marker,size)
        index = data1.query('label == @i').index.values
        ax.scatter(feature_reduced[index,0],feature_reduced[index,1],feature_reduced[index,2],c=c,marker=marker,s=size)
    plt.show()
    return None

#-------------------------------------------------------------------------------
def simulate(data,label,**argkw):
    data = data[data['label']==label]
    encoder = AS.dataset_simulator(**argkw)
    encoder.fit(np.array(data.drop('label',axis=1)))
    sim_data = pd.DataFrame(encoder.transform(),columns=data.columns.drop('label'))
    sim_data['label'] = label
    return sim_data
          
#-----------------------------------------------------------------------------
def subset_control(data,num):      ## randomly select a subset of certain ratio positive vs control samples 
    pos_index = data.query('label!=0').index.values
    control_sub_index = np.random.permutation(data.query('label==0').index.values)[:len(pos_index)*num]
    select_index = np.concatenate((pos_index,control_sub_index))
    return data.loc[select_index,:]
#------------------------------------------------------------------------------
def sparse_autoencoder_score(ae_params):
    global encoders
    sparse_ae = sac.sparse_autoencoder(**ae_params)
    score = commons.cross_validate_score(sparse_ae,reduced_train_x,train_label,ae_sample_weights_train)
    encoders.extend([sparse_ae])
    if math.isnan(score) or math.isfinite(score):
        score = np.Infinity
    return {'loss':-score,'status':STATUS_OK}
#-----------------------------------------------------------------
def selected_feature_analysis(features,X,y):
    t_stats = []
    pos = X[y==1]
    neg = X[y==0]
    for feature in features:
        test = TTest.FeatureTTest(feature)
        t_stats.extend([test.fit(pos,neg)])
    return pd.DataFrame(t_stats)
#--------------------------------------------------------------------
def method_params(methods=['random_forest','xgboost','logistic_regression','linear_SVC']):
    params={}
    feature_num = train_x.shape[1]
    #class_weight = {0:1,1:30}
    class_weight = None
    l_param={'C':9,'penalty':'l1'}
    rf_param = {'n_estimators':1000,'max_depth':10,'min_samples_split':14 ,'min_samples_leaf':1, 'n_jobs':-1 }
    svc_param = {'C':2,'dual':False,'penalty':'l1'}
    xgb_param = {'learning_rate':0.1,'max_depth':10,'n_estimators':1500,'reg_lambda':40,'gamma':1,'n_jobs':-1}
    mutual_information_param = {'k':100}
    fisher_param = {'k':100}
    if 'logistic_regression' in methods:
        params['logistic_regression'] = l_param
    if 'random_forest' in methods:
        params['random_forest'] = rf_param
    if 'linear_SVC' in methods:
        params['linear_SVC'] = svc_param
    if 'xgboost' in methods:
        params['xgboost'] = xgb_param
    if 'mutual_information' in methods:
        params['mutual_information'] = mutual_information_param
    if 'fisher_score' in methods:
        params['fisher_score'] = fisher_param   
    return params

In [3]:
dataset = 'AD_CpG'
log_dir = home+'logs/'
logger = Logger.Logger(log_dir,False).get_logger()
with pd.HDFStore(home+'data/'+dataset+'/all_features','r') as h5s:
    all_data = h5s['all_features']
all_data['beta_sign'] = all_data['label']
#all_data['coordinate'] = all_data['coordinate'].astype('i8')
all_data.drop(['coordinate','chr'],axis=1,inplace=True)
all_data['dist_to_nearest_tss'] = all_data['dist_to_nearest_tss'].astype('i8')
all_data = data_selection(all_data,classes=[0,1,-1],combine=True)
#all_data = data_selection(all_data,classes=[0,1,-1],combine=True)
all_features = all_data
#all_features = subset_control(all_data,30)
#all_features = all_data.query('beta_sign>0') ##only for hypermethylated sites in RICHS dataset, for AD dataset, hyper/hypo status can't be determined from beta
#logger.info('only keep heypermethylated sites')

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#split train test data and scaling on train data
scaler='MinMax'
all_features.drop(['id','winid','beta','beta_sign'],axis=1,inplace=True)
train_x,train_label,test_x,test_label = commons.train_test_split(all_features,scaler=scaler)
train_x.reset_index(drop=True,inplace=True)
train_label.reset_index(drop=True,inplace=True)
test_x.reset_index(drop=True,inplace=True)
test_label.reset_index(drop=True,inplace=True)
logger.info('Data scaler type is: %s',scaler)

In [5]:
sample_weights_train = commons.sample_weights(train_x,train_label,factor=1)
sample_weights_test = commons.sample_weights(test_x,test_label,factor=1)
weight_min_max_ratio = sample_weights_train.max()/sample_weights_train.min()
logger.info('weight max ratio: %f',weight_min_max_ratio)

In [6]:
weight_min_max_ratio

25.605507916274266

In [7]:
train_x.drop(['pvalue'],axis=1,inplace=True)
test_x.drop(['pvalue'],axis=1,inplace=True)

In [9]:
fs_sample_weights = np.power(sample_weights_train,0.5) if dataset == 'RICHS' else sample_weights_train
methods = ['random_forest','xgboost','logistic_regression','linear_SVC']
all_intersect = False
fs_params = method_params(methods)
fs = FS.FeatureSelection(class_num=2,methods=methods,all_intersect=all_intersect,**fs_params)
logger.info('Feature selection methods are: '+str(methods))
logger.info('All intersected features: '+str(all_intersect))
fs.fit(sample_weight=fs_sample_weights)
selected_features = fs.transform(train_x,train_label)
logger.info('selected features number is: %d\n',selected_features.shape[0])
logger.info(selected_features)
reduced_train_x = train_x[selected_features['feature']]
reduced_test_x = test_x[selected_features['feature']]
total_x = pd.concat([reduced_train_x,reduced_test_x],ignore_index=True)
total_label = pd.concat([train_label,test_label],ignore_index=True)
total_weights = pd.concat([sample_weights_train,sample_weights_test],ignore_index=True)

feature_diff_stats = selected_feature_analysis(selected_features['feature'],total_x,total_label)
feature_diff_stats = pd.merge(feature_diff_stats,selected_features,on='feature')
selected_features_100 = feature_diff_stats if len(feature_diff_stats) <=50 else feature_diff_stats.sort_values(['n','pvalue'],ascending=[False,True])[:60]
reduced_train_x = train_x[selected_features_100['feature']]
reduced_test_x = test_x[selected_features_100['feature']]
total_x = pd.concat([reduced_train_x,reduced_test_x],ignore_index=True)
total_label = pd.concat([train_label,test_label],ignore_index=True)
total_weights = pd.concat([sample_weights_train,sample_weights_test],ignore_index=True)

[[0. 0. 0. ... 0. 0. 0.]]


In [12]:
selected_features_100

,diff(pos-neg),feature,pvalue,tstats,n
227,0.102378,Left_Ventricle-H3K4me1,1.181308e-11,6.834098,4.0
224,0.107347,CD14_Primary_Cells-H3K4me1,3.079478e-10,6.336093,4.0
226,0.078766,Psoas_Muscle-H3K27ac,8.115685e-10,6.181129,4.0
225,0.092666,Adipose_Nuclei-H3K4me1,1.005927e-07,5.350910,4.0
222,0.087205,Fetal_Placenta-H3K4me1,2.215347e-07,5.203583,4.0
223,0.199831,genocanyon_score,7.461648e-07,4.969314,4.0
221,-0.029283,Bone_Marrow_Derived_Mesenchymal_Stem_Cell_Cult...,2.019461e-03,-3.092594,4.0
228,0.041643,NCFF435ETE_WGBS_counts,4.063273e-03,2.877491,4.0
206,0.117927,Monocytes-CD14+_RO01746-H3K4me1,3.113103e-12,7.028557,3.0
188,0.085339,Right_Atrium-H3K27ac,2.688508e-10,6.357532,3.0


In [17]:
feature_diff_stats.to_csv(home+'data/'+dataset+'/feature_stas.csv')

In [ ]:
#TSNEPlot(plot_data,class_labels=[0,1,2],param_map={0:('g','^',20),1:('b','*',20),2:('r','o',20)})
plot_data = reduced_train_x.copy()
plot_data['label'] = train_label
TSNEPlot(plot_data,class_labels=[0,1],param_map={0:('k','^',20),1:('r','*',20)})

In [18]:
with pd.HDFStore(home+'data/'+dataset+'/selected_features','w') as h5s:
    h5s['train_x'] = reduced_train_x
    h5s['train_label'] = train_label
    h5s['test_x'] = reduced_test_x
    h5s['test_label'] = test_label
    h5s['sample_weights_train'] = sample_weights_train
    h5s['sample_weights_test'] = sample_weights_test